In [1]:
import os

In [2]:
%pwd

'c:\\pawan chinnu data science\\Chest-Cancer-Detection\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\pawan chinnu data science\\Chest-Cancer-Detection'

In [5]:
os.environ["MLFLOW-TRACKING-URI"]="https://dagshub.com/pawanchinnu17/DeepScan-Automated-Chest-Cancer-Detection-Using-MLflow-and-DVC.mlflow"
os.environ["MLFLOW-TRACKING_USERNAME"]="pawanchinnu17"
os.environ["MLFLOW-TRACKING_PASSWORD"]="afa64adc979875924b96b19bd10958adf02a2ad1"